In [1]:
from functions import get_energy

In [2]:
df=get_energy.get()

  0%|          | 0/256 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 256/256 [00:31<00:00,  8.00it/s]


In [3]:
df.head()

,gesamt,weekday
date_time,,
2018-12-24 00:00:00,42.02925,0
2018-12-24 01:00:00,39.61025,0
2018-12-24 02:00:00,39.13875,0
2018-12-24 03:00:00,39.42100,0
2018-12-24 04:00:00,40.74775,0


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from tqdm import tqdm
from datetime import datetime, timedelta
from functions import get_energy

# Fetching and initial preprocessing of the dataset
df = get_energy.get()
df.head()

# Define the lead times
horizons_def = [36, 40, 44, 60, 64, 68]
horizons = [h + 1 for h in horizons_def]

# Function to calculate future dates based on a horizon
def get_date_from_horizon(last_ts, horizon):
    return last_ts + pd.DateOffset(hours=horizon)


# Adjusting the last date to the nearest Thursday
current_date = datetime.now()
days_until_thursday = 3 - current_date.weekday()
if days_until_thursday < 0:
    days_until_thursday += 7
thursday_of_current_week = current_date + timedelta(days=days_until_thursday)
thursday_of_current_week = thursday_of_current_week.replace(hour=0, minute=0, second=0, microsecond=0)
LAST_DATE = thursday_of_current_week

# Generating horizon dates
horizon_date = [get_date_from_horizon(LAST_DATE, h) for h in horizons]

# Define the quantiles for prediction
tau = [0.025, 0.25, 0.5, 0.75, 0.975]

# Preprocessing for regression
df['month'] = df.index.month
df['hour'] = df.index.hour
month_dummies = pd.get_dummies(df['month'], prefix='month', drop_first=True)
hour_dummies = pd.get_dummies(df['hour'], prefix='hour', drop_first=True)
df = df.join(month_dummies).join(hour_dummies)

# Implementing Quantile Regression
import statsmodels.api as sm

# Define the independent variables (X) and the dependent variable (y)
exclude_columns = ['gesamt', 'month', 'hour']
X = df[[col for col in df.columns if col not in exclude_columns]]
y = df['gesamt']

# Convert data types
X = X.apply(pd.to_numeric)
y = y.apply(pd.to_numeric)
X = X.astype(int)

# Add a constant to the model (for the intercept)
X = sm.add_constant(X)

# Train a model for each quantile
quantile_models = {quantile: sm.QuantReg(y, X).fit(q=quantile) for quantile in tau}

# Print summary of one of the quantile models (e.g., median model)
print(quantile_models[0.5].summary())

# Additional steps for visualization or output formatting can be added here

  0%|          | 0/256 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 256/256 [00:33<00:00,  7.62it/s]


                         QuantReg Regression Results                          
Dep. Variable:                 gesamt   Pseudo R-squared:               0.5067
Model:                       QuantReg   Bandwidth:                      0.6993
Method:                 Least Squares   Sparsity:                        11.95
Date:                Sun, 19 Nov 2023   No. Observations:                43003
Time:                        19:05:24   Df Residuals:                    42967
                                        Df Model:                           35
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         58.7920      0.175    336.249      0.000      58.449      59.135
weekday       -1.8483      0.014   -128.300      0.000      -1.877      -1.820
month_2        0.4336      0.142      3.055      0.002       0.155       0.712
month_3       -3.2361      0.139    -23.362      0.0

In [60]:
# Create a new DataFrame for the forecast
forecast_df = pd.DataFrame()
forecast_df = pd.DataFrame({'date_time': horizon_date})
# Extract and one-hot encode month and hour from the date_time
forecast_df['month'] = forecast_df['date_time'].dt.month
forecast_df['hour'] = forecast_df['date_time'].dt.hour
forecast_df = forecast_df.join(pd.get_dummies(forecast_df['month'], prefix='month', drop_first=True))
forecast_df = forecast_df.join(pd.get_dummies(forecast_df['hour'], prefix='hour', drop_first=True))

# Drop the original month and hour columns
forecast_df.drop(['month', 'hour'], axis=1, inplace=True)

# Add constant and weekday columns
forecast_df['const'] = 1.0
forecast_df['weekday'] = forecast_df['date_time'].dt.dayofweek

# Ensure all columns in X are also in forecast_df
for col in X.columns:
    if col not in forecast_df.columns and col != 'date_time':
        forecast_df[col] = 0

# Reorder columns to match X (excluding date_time)
forecast_columns = [col for col in X.columns if col != 'date_time']
forecast_df = forecast_df[['date_time'] + forecast_columns]
forecast_df.drop(['date_time'], axis=1, inplace=True)
forecast_df.head()

,const,weekday,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,1.0,4,0,0,0,0,0,0,0,0,...,0,0,0,False,0,0,0,False,0,0
1,1.0,4,0,0,0,0,0,0,0,0,...,0,0,0,True,0,0,0,False,0,0
2,1.0,4,0,0,0,0,0,0,0,0,...,0,0,0,False,0,0,0,True,0,0
3,1.0,5,0,0,0,0,0,0,0,0,...,0,0,0,False,0,0,0,False,0,0
4,1.0,5,0,0,0,0,0,0,0,0,...,0,0,0,True,0,0,0,False,0,0


In [61]:
quantile_models

{0.025: <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x11f605c70>,
 0.25: <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x11e56b340>,
 0.5: <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x11f5ecdc0>,
 0.75: <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x11f5eceb0>,
 0.975: <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x11f5ecc40>}

In [62]:
forecast_df2 = forecast_df.copy()

In [63]:


# Make predictions for each quantile
for quantile, model in quantile_models.items():
    forecast_var = model.predict(forecast_df)
    forecast_df2[f'prediction_q{quantile}'] = forecast_var

'''
# Formatting the output
df_sub = pd.DataFrame()
for quantile in tau:
    df_sub[f'q{quantile}'] = forecast_df[f'prediction_q{quantile}']
print(df_sub)
'''

"\n# Formatting the output\ndf_sub = pd.DataFrame()\nfor quantile in tau:\n    df_sub[f'q{quantile}'] = forecast_df[f'prediction_q{quantile}']\nprint(df_sub)\n"

In [64]:
forecast_df2

,const,weekday,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,...,hour_19,hour_20,hour_21,hour_22,hour_23,prediction_q0.025,prediction_q0.25,prediction_q0.5,prediction_q0.75,prediction_q0.975
0,1.0,4,0,0,0,0,0,0,0,0,...,0,0,False,0,0,39.669707,47.272252,51.398751,54.550855,57.751813
1,1.0,4,0,0,0,0,0,0,0,0,...,0,0,False,0,0,50.648201,59.9643,64.015125,68.887519,74.302375
2,1.0,4,0,0,0,0,0,0,0,0,...,0,0,True,0,0,48.918749,56.234627,59.336122,62.736017,66.78625
3,1.0,5,0,0,0,0,0,0,0,0,...,0,0,False,0,0,38.680794,45.395227,49.550439,53.117397,56.996875
4,1.0,5,0,0,0,0,0,0,0,0,...,0,0,False,0,0,49.659288,58.087275,62.166813,67.454061,73.547438
5,1.0,5,0,0,0,0,0,0,0,0,...,0,0,True,0,0,47.929836,54.357603,57.487809,61.302559,66.031313


In [65]:
from datetime import date
date_str = date.today()  #- timedelta(days=1)
date_str = date_str
date_str
df_sub = pd.DataFrame({
    "forecast_date": date_str,
    "target": "energy",
    "horizon": [str(h) + " hour" for h in horizons_def],
    "q0.025": forecast_df2["prediction_q0.025"],
    "q0.25": forecast_df2["prediction_q0.25"],
    "q0.5": forecast_df2["prediction_q0.5"],
    "q0.75": forecast_df2["prediction_q0.75"],
    "q0.975": forecast_df2["prediction_q0.975"]})
df_sub

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-19,energy,36 hour,39.669707,47.272252,51.398751,54.550855,57.751813
1,2023-11-19,energy,40 hour,50.648201,59.9643,64.015125,68.887519,74.302375
2,2023-11-19,energy,44 hour,48.918749,56.234627,59.336122,62.736017,66.78625
3,2023-11-19,energy,60 hour,38.680794,45.395227,49.550439,53.117397,56.996875
4,2023-11-19,energy,64 hour,49.659288,58.087275,62.166813,67.454061,73.547438
5,2023-11-19,energy,68 hour,47.929836,54.357603,57.487809,61.302559,66.031313


In [ ]:
X.head()